# APIs

In [3]:
from elasticsearch import Elasticsearch
import json
from pprint import pprint

In [4]:
es=Elasticsearch(hosts='http://localhost:9200/')
client_info=es.info()
print("Connected to Elastic-Search !")
pprint(client_info.body)

Connected to Elastic-Search !
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'XoT9Xm1xR3O6L3zfFdu5nQ',
 'name': 'd8153502b8b9',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-02-13T09:35:20.314882762Z',
             'build_flavor': 'default',
             'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.4.2',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.6.2'}}


In [20]:
es.indices.delete(index="test_index", ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})

In [21]:
res=es.indices.create(index="test_index")

In [22]:
docs=[
    {
        "test_field":"test content1"
    },
    {
        "test_field":"test content2"
    },
    {
        "test_field":"test content3"
    }
]

In [23]:
res_list=[]
for doc in docs:
    res= es.index(index="test_index", document=doc)
    res_list.append(res.body)

In [24]:
res_list

[{'_index': 'test_index',
  '_id': 'ml5vGJUBIxIS9hU_1qHR',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 0,
  '_primary_term': 1},
 {'_index': 'test_index',
  '_id': 'm15vGJUBIxIS9hU_16EU',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 1,
  '_primary_term': 1},
 {'_index': 'test_index',
  '_id': 'nF5vGJUBIxIS9hU_16Ec',
  '_version': 1,
  'result': 'created',
  '_shards': {'total': 2, 'successful': 1, 'failed': 0},
  '_seq_no': 2,
  '_primary_term': 1}]

## 1. Count

In [27]:
res=es.count(index="test_index")

In [28]:
pprint(res.body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'count': 3}


## 2. Exists

In [ ]:
# check if the index exists or not
res=es.indices.exists(index="test_index") # return boolean
pprint(res.body)

True


In [31]:
# check with invalid index
res=es.indices.exists(index="invalid_index") # return boolean
pprint(res.body)

False


In [33]:
# check if the document exists in the index or not

res=es.exists(index=res_list[0]["_index"], id=res_list[0]["_id"])
pprint(res.body)

True


# Update API

In [6]:
# delete the update_api_test_index index if already exists
es.indices.delete(index="update_api_test_index", ignore_unavailable=True)
# create an index
es.indices.create(index="update_api_test_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'update_api_test_index'})

In [7]:
docs=[
    {
        "test_field":"test content1"
    },
    {
        "test_field":"test content2"
    },
    {
        "test_field":"test content3"
    }
]

In [8]:
res_list=[]
for doc in docs:
    res_list.append(es.index(index="update_api_test_index", document=doc).body)

In [12]:
print("=> document count ::",es.count(index="update_api_test_index").body['count'])

=> document count :: 3


In [15]:
# use update api for updating the value of a record
res_update_field_value=es.update(
    index="update_api_test_index",
    id=res_list[0]["_id"],
    doc={
        "test_field":"updated_data"
    }
)

In [16]:
res_update_field_value.body

{'_index': 'update_api_test_index',
 '_id': '-qidNpUBUdIER6xM_7Gh',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 3,
 '_primary_term': 1}

In [18]:
es.get(index="update_api_test_index",id=res_list[0]["_id"]).body

{'_index': 'update_api_test_index',
 '_id': '-qidNpUBUdIER6xM_7Gh',
 '_version': 2,
 '_seq_no': 3,
 '_primary_term': 1,
 'found': True,
 '_source': {'test_field': 'updated_data'}}

In [28]:
# add new field using doc/ script parameter

new_field_added_to_doc_res=es.update(
    index="update_api_test_index",
    id=res_list[0]["_id"],
    doc={
        "new_field2":"hey this is new_field value"
    }
)

In [29]:
new_field_added_to_doc_res.body

{'_index': 'update_api_test_index',
 '_id': '-qidNpUBUdIER6xM_7Gh',
 '_version': 5,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 6,
 '_primary_term': 1}

In [23]:
es.get(index="update_api_test_index",id=res_list[0]["_id"]).body

{'_index': 'update_api_test_index',
 '_id': '-qidNpUBUdIER6xM_7Gh',
 '_version': 3,
 '_seq_no': 4,
 '_primary_term': 1,
 'found': True,
 '_source': {'test_field': 'updated_data',
  'new_field': 'hey this is new_field value'}}

In [25]:
new_field_value_update_res=es.update(
    index="update_api_test_index",
    id=res_list[0]["_id"],
    doc={
        "new_field":"hey this is new_field value 2 after update"
    }
)

In [26]:
new_field_value_update_res.body

{'_index': 'update_api_test_index',
 '_id': '-qidNpUBUdIER6xM_7Gh',
 '_version': 4,
 'result': 'noop',
 '_shards': {'total': 0, 'successful': 0, 'failed': 0},
 '_seq_no': 5,
 '_primary_term': 1}

In [30]:
es.get(index="update_api_test_index",id=res_list[0]["_id"]).body

{'_index': 'update_api_test_index',
 '_id': '-qidNpUBUdIER6xM_7Gh',
 '_version': 5,
 '_seq_no': 6,
 '_primary_term': 1,
 'found': True,
 '_source': {'test_field': 'updated_data',
  'new_field': 'hey this is new_field value 2 after update',
  'new_field2': 'hey this is new_field value'}}

In [32]:
# perform upsert operation in case of document does not exists

upsert_res=es.update(
    index="update_api_test_index",
    id="1",
    doc={
        "test_field":"hey this is the test field values for upsert using update"
    },
    doc_as_upsert=True
)

In [33]:
upsert_res.body

{'_index': 'update_api_test_index',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 7,
 '_primary_term': 1}

In [35]:
es.count(index="update_api_test_index").body["count"]

4

In [38]:
es.get(index="update_api_test_index", id="1").body

{'_index': 'update_api_test_index',
 '_id': '1',
 '_version': 1,
 '_seq_no': 7,
 '_primary_term': 1,
 'found': True,
 '_source': {'test_field': 'hey this is the test field values for upsert using update'}}